# Testing notebook

In [11]:
import xlsxwriter
import pandas as pd
import numpy as np

from classification_power_predictor import classification_power_predictor
from writing import *

## We need some testing data

In [12]:
# frame size
n = 1000

test_frame = pd.DataFrame({"numeric_variable": np.random.normal(10, 5, n), 
                           "object_variable": np.round(np.random.uniform(0, 10, n)).astype('O'),
                           "column with soo000oo000oo000oo000oo long name": np.random.normal(10, 5, n)})

nv = test_frame["numeric_variable"]
ov = test_frame["object_variable"]

probs1 = nv.apply(lambda x: (x/(max(nv) - min(nv))) + ((np.random.rand())/5))
probs2 = ov.apply(lambda x: (x/(max(ov) - min(ov))) + ((np.random.rand())/5))

f_probs = (probs1 + probs2)/2
f_probs[f_probs >= 1] = 1


Y = np.zeros(n)
Y[f_probs > 0.4] = 1
Y[f_probs > 0.7] = 2

Y = pd.Series(Y)
Y_strs = Y.replace({0: 'cetegory 0', 1:'category 1', 2:'category 2'})
Y.value_counts()
Y_binary = Y.replace({2:1})

na_containts_frame = test_frame.copy()

for col in test_frame.columns:
    na_containts_frame.loc[na_containts_frame.sample(int(n/50)).index, col] = np.NaN

## Computions funcitons

In [13]:
from computions import *

### get_describe_nominal function test

Basic situation

In [14]:
get_describe_nominal(test_frame.iloc[:,1], Y)

,count,1.0,1.0%,2.0,2.0%,0.0,0.0%
object_variable,,,,,,,
6.0,122,115,94.262295,3.0,2.459016,4.0,3.278689
7.0,104,85,81.730769,17.0,16.346154,2.0,1.923077
8.0,101,79,78.217822,22.0,21.782178,0.0,0.000000
4.0,101,74,73.267327,0.0,0.000000,27.0,26.732673
1.0,100,8,8.000000,0.0,0.000000,92.0,92.000000
2.0,100,31,31.000000,0.0,0.000000,69.0,69.000000
3.0,99,44,44.444444,0.0,0.000000,55.0,55.555556
9.0,87,41,47.126437,46.0,52.873563,0.0,0.000000
5.0,87,74,85.057471,1.0,1.149425,12.0,13.793103


Data frame with emptys

In [15]:
test = get_describe_nominal(na_containts_frame.iloc[:,1], Y)
print("Is all obs. are in table - ",  str(test['count'].sum() == n))
test = get_describe_nominal(na_containts_frame.fillna('empty').iloc[:,1], Y)
print("Is all obs. are in table - ",  str(test['count'].sum() == n))

Is all obs. are in table -  False
Is all obs. are in table -  True


### AUC computing test

AUC numeric - simple fucntion

In [16]:
print(get_AUC_numeric(test_frame['numeric_variable'] ,Y))
print(get_AUC_numeric(test_frame['numeric_variable'], Y_strs))
print(get_AUC_numeric(na_containts_frame['numeric_variable'], Y_strs))
print(get_AUC_numeric(test_frame['numeric_variable'], Y_binary))

{1.0: 0.5607188170993559, 2.0: 0.7831777282599498, 0.0: 0.2912200208550574}
{'category 1': 0.5607188170993559, 'category 2': 0.7831777282599498, 'cetegory 0': 0.2912200208550574}
{'category 1': 0.5601017488076311, 'category 2': 0.7811062173109442, 'cetegory 0': 0.2929950486663343}
{1.0: 0.7087799791449427, 0.0: 0.2912200208550574}


AUC nominal

In [17]:
non_table = get_AUC_nominal(test_frame['object_variable'], Y)
dn_tab = get_describe_nominal(test_frame.iloc[:,1], Y_strs)
with_table = get_AUC_nominal(test_frame['object_variable'], Y_strs, dn_tab)
print(with_table)

print('AUC bynary')
print(get_AUC_nominal(test_frame['object_variable'], Y_binary))

print('AUC with emptys - needs to be replaces in previous steps')
print(get_AUC_nominal(na_containts_frame['object_variable'].fillna('empty'), Y_strs))

get_AUC_nominal(na_containts_frame['object_variable'].fillna('empty'), Y_strs, 
                get_describe_nominal(na_containts_frame.iloc[:,1].fillna('empty'), Y_strs))


{'category 1': 0.8435263354805127, 'category 2': 0.9265036339190854, 'cetegory 0': 0.9376178890047502}
AUC bynary
{1.0: 0.9376178890047504, 0.0: 0.9376178890047502}
AUC with emptys - needs to be replaces in previous steps
{'category 1': 0.8427211432103584, 'category 2': 0.9243881570906082, 'cetegory 0': 0.9350503997219326}


{'category 1': 0.8427211432103584,
 'category 2': 0.9243881570906082,
 'cetegory 0': 0.9350503997219326}

Get publish AUC for any predictor type

In [18]:
non_table = get_full_AUC(test_frame['object_variable'], Y, 'nominal')
print('nominal predictor without table')
print(non_table)

with_table = get_full_AUC(test_frame['object_variable'], Y_strs, 'nominal', descr_table = dn_tab)
print('moninal predictor with table')
print(with_table)

numeric_var = get_full_AUC(test_frame['numeric_variable'], Y, 'numeric')
print('numeric predictor')
print(numeric_var)

nominal predictor without table
{1.0: {'AUC': 0.8435263354805127, 'rel_type': 1, 'GINI': 0.6870526709610254}, 2.0: {'AUC': 0.9265036339190854, 'rel_type': 1, 'GINI': 0.8530072678381708}, 0.0: {'AUC': 0.9376178890047502, 'rel_type': 1, 'GINI': 0.8752357780095004}}
moninal predictor with table
{'category 1': {'AUC': 0.8435263354805127, 'rel_type': 1, 'GINI': 0.6870526709610254}, 'category 2': {'AUC': 0.9265036339190854, 'rel_type': 1, 'GINI': 0.8530072678381708}, 'cetegory 0': {'AUC': 0.9376178890047502, 'rel_type': 1, 'GINI': 0.8752357780095004}}
numeric predictor
{1.0: {'AUC': 0.5607188170993559, 'rel_type': 1, 'GINI': 0.12143763419871179}, 2.0: {'AUC': 0.7831777282599498, 'rel_type': 1, 'GINI': 0.5663554565198996}, 0.0: {'AUC': 0.7087799791449426, 'rel_type': -1, 'GINI': 0.41755995828988524}}


Decoding AUC info as DataFrame

In [19]:
numeric_var = get_full_AUC(test_frame['numeric_variable'], Y, 'numeric')
AUC_info_to_DataFrame(numeric_var,'numeric_variable')

1.0                          2.0                     \
                       AUC rel_type      GINI       AUC rel_type      GINI   
numeric_variable  0.560719        1  0.121438  0.783178        1  0.566355   

                      0.0                    
                      AUC rel_type     GINI  
numeric_variable  0.70878       -1  0.41756

get_all_computions funcion test

In [20]:
print('describe_table with no nas')
print(get_all_comuptions(test_frame['object_variable'], Y)['describe_table'])
print('describe_table with nas')
print(get_all_comuptions(na_containts_frame['object_variable'], Y)['describe_table'])

describe_table with no nas
                 count  1.0       1.0%   2.0       2.0%   0.0       0.0%
object_variable                                                         
6.0                122  115  94.262295   3.0   2.459016   4.0   3.278689
7.0                104   85  81.730769  17.0  16.346154   2.0   1.923077
8.0                101   79  78.217822  22.0  21.782178   0.0   0.000000
4.0                101   74  73.267327   0.0   0.000000  27.0  26.732673
1.0                100    8   8.000000   0.0   0.000000  92.0  92.000000
2.0                100   31  31.000000   0.0   0.000000  69.0  69.000000
3.0                 99   44  44.444444   0.0   0.000000  55.0  55.555556
9.0                 87   41  47.126437  46.0  52.873563   0.0   0.000000
5.0                 87   74  85.057471   1.0   1.149425  12.0  13.793103
0.0                 56    2   3.571429   0.0   0.000000  54.0  96.428571
10.0                43   11  25.581395  32.0  74.418605   0.0   0.000000
describe_table with nas


Getting indicators from computions uotput

In [21]:
comp_result =  get_all_comuptions(test_frame['numeric_variable'], Y)
print('full numeric data')
get_predictor_row(comp_result)
print('numeric data with nas, with str categoryes')
comp_result =  get_all_comuptions(na_containts_frame['numeric_variable'], Y_strs)
get_predictor_row(comp_result)

full numeric data
numeric data with nas, with str categoryes


category 1                    category 2                     \
                        AUC rel_type      GINI        AUC rel_type      GINI   
numeric_variable   0.560102        1  0.120203   0.781106        1  0.562212   

                 cetegory 0                   Emptys Emptys part  
                        AUC rel_type     GINI                     
numeric_variable   0.707005       -1  0.41401     20        0.02

## Writing to excel

Adding a new sheet

In [22]:
xl_writer = pd.ExcelWriter("test_result/sheet_creation.xlsx",engine='xlsxwriter')

for i in range(20):
    add_sheet(xl_writer, 'some name')
    add_sheet(xl_writer, 'some very very very long long name')
xl_writer.close()

Adding a info about different predictors

In [23]:
my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()

Let's show final table

In [24]:
my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
#my_cpp.get_predictors_data()
my_cpp.result_DF

1.0                     \
                                                    AUC rel_type      GINI   
numeric_variable                               0.560719        1  0.121438   
object_variable                                0.843526        1  0.687053   
column with soo000oo000oo000oo000oo long name  0.528922       -1  0.057844   

                                                    2.0                     \
                                                    AUC rel_type      GINI   
numeric_variable                               0.783178        1  0.566355   
object_variable                                0.926504        1  0.853007   
column with soo000oo000oo000oo000oo long name  0.529283        1  0.058566   

                                                    0.0                     \
                                                    AUC rel_type      GINI   
numeric_variable                                0.70878       -1   0.41756   
object_variable                                0.937618        1  0.875236   
column with soo000oo000oo000oo000oo long name  0.518526        1  0.037052   

                                              Emptys Emptys part  
                                                                  
numeric_variable                                   0         0.0  
object_variable                                    0         0.0  
column with soo000oo000oo000oo000oo long name      0         0.0

Writig a double header table

In [25]:


xl_writer = pd.ExcelWriter("test_result/double_header_saver.xlsx",engine='xlsxwriter')
ws = xl_writer.book.add_worksheet('test')
xl_writer.sheets['test'] = ws      

my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
my_cpp.my_writer = xl_writer

print_double_column_header(  my_cpp, ws, 
                            my_cpp.result_DF.loc[['numeric_variable'],:],
                            "A1", 'Hello World', 
                            xl_writer.book.add_format(my_cpp.default_header_format))

xl_writer.close()

Writing info about some predictor

In [26]:
xl_writer = pd.ExcelWriter("test_result/writing_a_value.xlsx",engine='xlsxwriter')

ws = xl_writer.book.add_worksheet('test')
xl_writer.sheets['test'] = ws                             

my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
my_cpp.my_writer = xl_writer

default_predictor_printer(my_cpp, ws, my_cpp.get_predictors_data()['numeric_variable'], 
{'header_format':xl_writer.book.add_format(my_cpp.default_header_format),
  'desc_format':xl_writer.book.add_format(my_cpp.default_header_format),
  'class_ab_format':xl_writer.book.add_format(my_cpp.default_header_format)})

xl_writer.close()

In [27]:
xl_writer = pd.ExcelWriter("test_result/result_test.xlsx",engine='xlsxwriter')
my_cpp.write_to_book(xl_writer)
xl_writer.close()